In [1]:
import csv
import time
import requests
import openai
import os
import time
import csv
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, accuracy_score

In [14]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
print(api_key)

None


In [4]:
def evaluate_predictions(actual_df, prediction_df, label_col="label", pred_col="Predicted Fit", review_col="ReviewText"):
    """
    Evaluate the performance of predicted labels against actual labels.

    Args:
        actual_df (pd.DataFrame): DataFrame containing the actual labels with the review and label columns.
        prediction_df (pd.DataFrame): DataFrame containing the predicted labels with the review and predicted label columns.
        label_col (str): Column name in actual_df containing true labels.
        pred_col (str): Column name in prediction_df containing predicted labels.
        review_col (str): Column name used to merge the two DataFrames.

    Returns:
        dict: A dictionary containing accuracy, precision, recall, F1 score, and a detailed classification report.
    """
    # Combine the actual and predicted DataFrames on the review column
    comparison_df = pd.merge(
        actual_df[[review_col, label_col]], 
        prediction_df[[review_col, pred_col]], 
        on=review_col, 
        how="inner"
    )

    if comparison_df.empty:
        raise ValueError("No matching reviews found between actual and predicted data.")

    # Extract true and predicted labels
    y_true = comparison_df[label_col]
    y_pred = comparison_df[pred_col]

    # Calculate evaluation metrics
    metrics = {
        "accuracy": accuracy_score(y_true, y_pred) * 100,
        "precision": precision_score(y_true, y_pred, average="macro", zero_division=0) * 100,
        "recall": recall_score(y_true, y_pred, average="macro", zero_division=0) * 100,
        "f1_score": f1_score(y_true, y_pred, average="macro", zero_division=0) * 100,
        "classification_report": classification_report(y_true, y_pred, output_dict=True),
    }

    # Print classification report (optional)
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))

    return metrics

In [7]:

# Replace with your OpenAI API key
# Load the API key from the environment variable
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# API endpoint for testing
url = "https://api.openai.com/v1/models"

# Set headers with the API key
headers = {
    "Authorization": f"Bearer {OPENAI_API_KEY}",
    "Content-Type": "application/json"
}

# Send the test request
response = requests.get(url, headers=headers)

# Print the response
if response.status_code == 200:
    print("API key is valid. Models available:")
    print(response.json())
elif response.status_code == 401:
    print("Invalid API key. Please check and update your API key.")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

Invalid API key. Please check and update your API key.


## Fit Zero Shot

In [21]:
import pandas as pd
import requests
import time
from sklearn.metrics import accuracy_score

def classify_review(review_text, max_retries=10, wait_time=30):
    """
    Classifies a review using the OpenAI API with retries for handling rate limits.

    Args:
    - review_text (str): The text of the review to classify.
    - max_retries (int): Maximum number of retries if the API call fails.
    - wait_time (int): Time to wait (in seconds) between retries.

    Returns:
    - str: The raw response from the OpenAI API.
    """
    messages = [
        {
            "role": "system",
            "content": "You are an analyst here to classify product reviews into user-specified categories. Classify the following product reviews into one of the categories: \"Correct Size\", \"Wrong Size\" or \"No Comment\". Respond only with the category name."
        },
        {
            "role": "user",
            "content": f"""Classify this review: "{review_text}"."""
        }
    ]

    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "gpt-4",  # Use "gpt-4" if you have access; change to "gpt-3.5-turbo" if needed
        "messages": messages,
        "temperature": 0
    }

    for attempt in range(max_retries):
        response = requests.post(url, headers=headers, json=payload)

        if response.status_code == 429:  # Too Many Requests (rate limiting)
            print(f"Attempt {attempt + 1}: Rate limit hit. Waiting {wait_time} seconds before retrying...")
            time.sleep(wait_time)
        elif response.status_code == 200:  # Successful response
            break
        else:
            print(f"Attempt {attempt + 1}: Error {response.status_code}: {response.text}")
            return "Error"

    if response.status_code != 200:
        print(f"Error: {response.status_code}, {response.text}")
        return "Error"

    # Parse the output
    output = response.json()
    result = output["choices"][0]["message"]["content"].strip() if "choices" in output else "Error"
    print(result)
    return result  # Return the raw response from the API

In [22]:
input_file = "fit.csv"
fit = pd.read_csv(input_file)

# Apply cleaning to convert raw responses to valid categories
fit["Predicted Fit"] = fit["ReviewText"].apply(classify_review)

# Save the cleaned predictions to a file
output_file = "gpt_results/fit_predictions_zero_shot.csv"
fit.to_csv(output_file, index=False)

No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
"Wrong Size"
No Comment
No Comment
No Comment
No Comment
No Comment
Correct Size
No Comment
No Comment
No Comment
No Comment
No Comment
Correct Size
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
Correct Size
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
Correct Size
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
Wrong Size
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
No Comment
"Wrong Size"
No Comment
No Comment
No Comment
No Comment
No Comment
"Wrong Size"
Correct Size
No Comment
No Comment
No Co

In [5]:
input_file = "fit.csv"  # Original file with actual labels
predictions_file = "gpt_results/fit_predictions_zero_shot.csv"  # File with model predictions

# Load input and prediction files as DataFrames
df_input = pd.read_csv(input_file)
df_predictions = pd.read_csv(predictions_file)

# Check unique values in the 'label' column
unique_labels = df_predictions['Predicted Fit'].unique()
print(f"Unique labels: {unique_labels}")

# Count occurrences of each label
label_counts = df_predictions['Predicted Fit'].value_counts()
print(label_counts)

label_mapping = {
    '"Wrong Size"': 'Wrong Size',
    '"No Comment"': 'No Comment',
    '"Correct Size': 'Correct Size',
    'Error': 'No Comment'
}

# Replace incorrect labels with standardized ones
df_predictions['Predicted Fit'] = df_predictions['Predicted Fit'].replace(label_mapping)

# Evaluate predictions
metrics = evaluate_predictions(df_input, df_predictions)

# Print evaluation metrics
print(f"Accuracy: {metrics['accuracy']:.2f}%")
print(f"Precision: {metrics['precision']:.2f}%")
print(f"Recall: {metrics['recall']:.2f}%")
print(f"F1 Score: {metrics['f1_score']:.2f}")

Unique labels: ['No Comment' '"Wrong Size"' 'Correct Size' 'Wrong Size' '"No Comment"'
 '"Correct Size"' 'Error']
Predicted Fit
No Comment        1725
Correct Size       311
"Wrong Size"        95
Wrong Size          73
"No Comment"        32
"Correct Size"      18
Error                1
Name: count, dtype: int64

Classification Report:
                precision    recall  f1-score   support

"Correct Size"       0.00      0.00      0.00         0
  Correct Size       0.74      0.35      0.47       659
    No Comment       0.71      0.91      0.80      1375
    Wrong Size       0.74      0.57      0.64       221

      accuracy                           0.71      2255
     macro avg       0.55      0.46      0.48      2255
  weighted avg       0.72      0.71      0.69      2255

Accuracy: 71.44%
Precision: 54.95%
Recall: 45.70%
F1 Score: 47.97


/home/anand173/.conda/envs/cent7/2024.02-py311/llama2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/anand173/.conda/envs/cent7/2024.02-py311/llama2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/anand173/.conda/envs/cent7/2024.02-py311/llama2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera